# cis550 final project data cleaning

Data cleaning done by Estee and Catherine.

Main steps taken for data cleaning:



In [ ]:
! pip install -U pandasql
import numpy as np
import pandas as pd
import pandasql as ps


  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26820 sha256=54fb194dd1f9b3a07d73d0a8abdebe7f6b4651bf15005ab05400b290a0c0c453
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import data

grdf = pd.read_csv('/content/drive/My Drive/CIS550 Final Project/goodreads_book_data.csv', header=0)
grdf


,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
...,...,...,...,...,...,...,...,...,...,...,...,...
54296,Howard Megdal,"In this fearless and half-crazy story, Howard ...",NaN,Hardcover,9.78161E+12,256 pages,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,Sports|Baseball|Sports and Games|Sports|Nonfic...,https://images.gr-assets.com/books/1312074392l...
54297,Howard Megdal,From the icons of the game to the players who ...,NaN,Hardcover,9.78006E+12,256 pages,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",Nonfiction|Sports and Games|Sports,https://images.gr-assets.com/books/1348841629l...
54298,Howard Megdal,NaN,NaN,Kindle Edition,NaN,NaN,3.66,32,3,"Wilpon's Folly - The Story of a Man, His Fortu...",Sports|Baseball|Abandoned,https://images.gr-assets.com/books/1394277097l...
54299,Mimi Baird|Eve Claxton,"Soon to be a major motion picture, from Brad P...",NaN,Hardcover,9.7808E+12,272 pages,3.82,867,187,He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,https://images.gr-assets.com/books/1403192135l...


In [ ]:
grdf.columns

Index(['book_authors', 'book_desc', 'book_edition', 'book_format', 'book_isbn',
       'book_pages', 'book_rating', 'book_rating_count', 'book_review_count',
       'book_title', 'genres', 'image_url'],
      dtype='object')

## Part 1: Renaming column names to match ER diagram

In [ ]:
grdf_cleaned = grdf.drop(['book_edition', 'book_format', 'book_isbn'], axis=1)


In [ ]:
# ADD ID COLUMN

id = pd.Series(range(1, len(grdf_cleaned) + 1))

grdf_cleaned['id'] = id



In [ ]:
# rename columns

grdf_cleaned.columns = ['authors', 'description', 
       'page_num', 'rating', 'rating_num', 'review_num',
       'title', 'genres', 'image_url', 'id']



In [ ]:
grdf_cleaned

,authors,description,page_num,rating,rating_num,review_num,title,genres,image_url,id
0,Suzanne Collins,Winning will make you famous. Losing means cer...,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...,1
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...,2
2,Harper Lee,The unforgettable novel of a childhood in a sl...,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...,3
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...,4
4,Stephenie Meyer,About three things I was absolutely positive.F...,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...,5
...,...,...,...,...,...,...,...,...,...,...
54296,Howard Megdal,"In this fearless and half-crazy story, Howard ...",256 pages,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,Sports|Baseball|Sports and Games|Sports|Nonfic...,https://images.gr-assets.com/books/1312074392l...,54297
54297,Howard Megdal,From the icons of the game to the players who ...,256 pages,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",Nonfiction|Sports and Games|Sports,https://images.gr-assets.com/books/1348841629l...,54298
54298,Howard Megdal,NaN,NaN,3.66,32,3,"Wilpon's Folly - The Story of a Man, His Fortu...",Sports|Baseball|Abandoned,https://images.gr-assets.com/books/1394277097l...,54299
54299,Mimi Baird|Eve Claxton,"Soon to be a major motion picture, from Brad P...",272 pages,3.82,867,187,He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,https://images.gr-assets.com/books/1403192135l...,54300


Save the joined df into a new file, and that's the main "books" file we need

## Part 2: Genre Separation and Indexing

###2.1
take out the genre column of goodreads dataset and the id, and seperates all genres into individual rows with the respective book id. 

In [ ]:
# get separate df for genres
query = """
select id, genres
from grdf_cleaned
"""

genres_original_df = ps.sqldf(query)


In [ ]:
# split genres into multiple rows

genre_df = pd.DataFrame(columns=['book_id', 'genre'])

# fill na with ""
genres_original_df['genres'] = genres_original_df['genres'].fillna("")


In [ ]:
genres_original_df

,id,genres
0,1,Young Adult|Fiction|Science Fiction|Dystopia|F...
1,2,Fantasy|Young Adult|Fiction
2,3,Classics|Fiction|Historical|Historical Fiction...
3,4,Classics|Fiction|Romance
4,5,Young Adult|Fantasy|Romance|Paranormal|Vampire...
...,...,...
54296,54297,Sports|Baseball|Sports and Games|Sports|Nonfic...
54297,54298,Nonfiction|Sports and Games|Sports
54298,54299,Sports|Baseball|Abandoned
54299,54300,Nonfiction|Autobiography|Memoir|Biography|Psyc...


In [ ]:

genre_df = pd.concat([pd.Series(row['id'], row['genres'].split('|'))              
                    for _, row in genres_original_df.iterrows()]).reset_index()

In [ ]:
# print

genre_df.columns = ['genre', 'book_id']
genre_df

,genre,book_id
0,Young Adult,1
1,Fiction,1
2,Science Fiction,1
3,Dystopia,1
4,Fantasy,1
...,...,...
303123,Literature,54301
303124,American,54301
303125,Literature,54301
303126,20th Century,54301


### 2.2
assign id to unique genres

In [ ]:
# get unique genre names and assign ids to it
query = """
select distinct genre as genre_name
from genre_df
"""

genre_id_df = ps.sqldf(query)

# add index col
genre_id_df['genre_id'] = pd.Series(range(1, len(genre_id_df) + 1))

In [ ]:
# print
print(len(genre_id_df))
genre_id_df

867


,genre_name,genre_id
0,Young Adult,1
1,Fiction,2
2,Science Fiction,3
3,Dystopia,4
4,Fantasy,5
...,...,...
862,Peak Oil,863
863,Social Change,864
864,Health Care,865
865,Read For College,866


In [ ]:
# save the genre dataset
genre_id_df.to_csv('/content/drive/My Drive/CIS550 Final Project/genres.csv')

###2.3
 replace the genre names in genre_df to genre ids

 save book_has_genre

In [ ]:
# replace the genre names in genre_df with genre id

query = """
select genre_df.book_id, genre_id_df.genre_id from 
genre_df left join genre_id_df on genre_df.genre = genre_id_df.genre_name
"""

# schema: book_id, genre_id
book_has_genre = ps.sqldf(query)

In [ ]:
book_has_genre

,book_id,genre_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
303123,54301,15
303124,54301,43
303125,54301,15
303126,54301,75


In [ ]:
# save the bookHasGenre dataset
book_has_genre.to_csv('/content/drive/My Drive/CIS550 Final Project/book_has_genre.csv')

## Part 3: Author data cleaning



### 3.1
split the authors in grdf.book_authors (similar to genre-splitting)

In [ ]:

query = """
select id, authors
from grdf_cleaned
"""

authors_original_df = ps.sqldf(query)

### 3.2

construct an authors df from the authors from 3.1, add an index column, and then replace the author_name in book_author with author_id.

save book_author and authors

In [ ]:
# split genres into multiple rows

authors_book_df = pd.DataFrame(columns=['book_id', 'author_name'])


In [ ]:

# for _, row in authors_original_df.iterrows():
#   if pd.isnull(row['authors']):
#     authors_book_df = genre_df.append([row['id'], ""])
#     continue
#   else:
#     for string in row['authors'].split('|'):
#       authors_book_df = genre_df.append([row['id'], string])
    
authors_book_df = pd.concat([pd.Series(row['id'], row['authors'].split('|'))              
                    for _, row in authors_original_df.iterrows()]).reset_index()

In [ ]:
# print
authors_book_df.columns = ['author_name', 'book_id']
authors_book_df

,author_name,book_id
0,Suzanne Collins,1
1,J.K. Rowling,2
2,Mary GrandPré,2
3,Harper Lee,3
4,Jane Austen,4
...,...,...
73174,Howard Megdal,54298
73175,Howard Megdal,54299
73176,Mimi Baird,54300
73177,Eve Claxton,54300


In [ ]:
# get unique author names and assign ids to it
query = """
select distinct author_name
from authors_book_df
"""

authors = ps.sqldf(query)

# add index col
authors['author_id'] = pd.Series(range(1, len(authors) + 1))

In [ ]:
# print
print(len(authors))
authors

32041


,author_name,author_id
0,Suzanne Collins,1
1,J.K. Rowling,2
2,Mary GrandPré,3
3,Harper Lee,4
4,Jane Austen,5
...,...,...
32036,Avi Steinberg,32037
32037,Howard Megdal,32038
32038,Mimi Baird,32039
32039,Eve Claxton,32040


In [ ]:
# save the authors dataset
authors.to_csv('/content/drive/My Drive/CIS550 Final Project/authors_no_info_processed.csv')

In [ ]:
# replace the authors names in author_book_df with author id

query = """
select authors_book_df.book_id, authors.author_id from 
authors_book_df left join authors on authors_book_df.author_name = authors.author_name
"""

# schema: book_id, author_id
book_author = ps.sqldf(query)

In [ ]:
book_author

,book_id,author_id
0,1,1
1,2,2
2,2,3
3,3,4
4,4,5
...,...,...
73174,54298,32038
73175,54299,32038
73176,54300,32039
73177,54300,32040


In [ ]:
# save the bookHasGenre dataset
book_author.to_csv('/content/drive/My Drive/CIS550 Final Project/book_author.csv')

### 3.3 

additional author data from another dataset

In [ ]:
authors_raw_df = pd.read_csv('/content/drive/My Drive/CIS550 Final Project/authors_raw_data.csv', header=0)
authors_no_info = pd.read_csv('/content/drive/My Drive/CIS550 Final Project/authors_no_info_processed.csv', header=0)
print(len(authors_raw_df))

22891


In [ ]:
# check how many distinct authors
len(pd.unique(authors_raw_df['author_name']))
authors_raw_df.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

drop some columns

In [ ]:

authors_raw_df = authors_raw_df.drop(['author_rating_count','author_review_count','book_fullurl', 'book_id', 'book_title','num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'], axis=1)
print(authors_raw_df.columns)


Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'birthplace', 'book_average_rating',
       'genre_1', 'genre_2'],
      dtype='object')


In [ ]:
authors_raw_df['author_name'] = authors_raw_df['author_name'].apply(lambda x:  x.split('\n')[0])

In [ ]:
authors_raw_df['birthplace'] = authors_raw_df['birthplace'].apply(lambda x:  x.split('\n')[0])

In [ ]:
authors_raw_df = authors_raw_df.drop(['genre_1', 'genre_2', 'book_average_rating', 'author_id'], axis=1)

In [ ]:
# merge with authors as much as possible
query = """select distinct *
from authors_no_info a1 left join authors_raw_df a2 on a1.author_name = a2.author_name
"""

authors_with_info = ps.sqldf(query)


In [ ]:
authors_with_info = authors_with_info.drop(['Unnamed: 0'], axis=1)
authors_with_info.columns = ['author_name', 'author_id', 'ave_rating',
       'gender', 'author_genres', 'author_name2', 'page_url',
       'birthplace']
authors_with_info = authors_with_info.drop(['author_name2'], axis=1)

In [ ]:
# swap columns
columns_titles = ['author_id', 'author_name', 'ave_rating', 'gender', 'author_genres', 'page_url','birthplace']
authors_with_info = authors_with_info.reindex(columns=columns_titles)
authors_with_info.columns

Index(['author_id', 'author_name', 'ave_rating', 'gender', 'author_genres',
       'page_url', 'birthplace'],
      dtype='object')

In [ ]:
# save the final data
authors_with_info.to_csv('/content/drive/My Drive/CIS550 Final Project/authors.csv')

### 3.4* author genres

dealing with author genres, adding some other unique genres to "genres"


In [ ]:
# a sepaate df for author and genre
query = """
select author_id, author_genres
from authors_with_info"""

author_genres_df = ps.sqldf(query)
author_genres_df

,author_id,author_genres
0,1,"fiction,science-fiction-fantasy,"
1,2,"fiction,"
2,3,None
3,4,"literature-fiction,"
4,5,"fiction,romance,"
...,...,...
32090,32037,None
32091,32038,None
32092,32039,None
32093,32040,None


In [ ]:
# separate genres into individual lines
author_genres_df['author_genres']  = author_genres_df['author_genres'].fillna("")
author_genre_signle =  pd.concat([pd.Series(row['author_id'], row['author_genres'].split(','))        
                    for _, row in author_genres_df.iterrows()]).reset_index()

author_genre_signle.columns=['author_genre', 'author_id']


In [ ]:
author_genre_signle

,author_genre,author_id
0,fiction,1
1,science-fiction-fantasy,1
2,,1
3,fiction,2
4,,2
...,...,...
41749,,32037
41750,,32038
41751,,32039
41752,,32040


In [ ]:
genres_cleaned = pd.read_csv('/content/drive/My Drive/CIS550 Final Project/genres.csv', header=0)

In [ ]:
# cleaning genres again
genres_cleaned = genres_cleaned.drop(['Unnamed: 0'], axis=1)
columns_titles = ['genre_id', 'genre_name']
genres_cleaned = genres_cleaned.reindex(columns=columns_titles)

In [ ]:
# convert all id to lower case
genres_cleaned['genre_name'] = genres_cleaned['genre_name'].apply(lambda x: str(x).lower())
genres_cleaned

,genre_id,genre_name
0,1,young adult
1,2,fiction
2,3,science fiction
3,4,dystopia
4,5,fantasy
...,...,...
862,863,peak oil
863,864,social change
864,865,health care
865,866,read for college


In [ ]:
# find the genres not in genres_cleaned and add to it
query = """
select distinct author_genre from
author_genre_signle
where author_genre not in (select genre_name from genres_cleaned) and author_genre <> ""
"""

temp = ps.sqldf(query)

id_from_868 = pd.Series(range(868, 868+len(temp)-1 + 1))
df = pd.concat([id_from_868, temp['author_genre']], axis=1)
df.columns = ['genre_id', 'genre_name']

query2 = """
select * from
(select * from genres_cleaned 
 union 
 select * from df)
  c
"""

added_genres = ps.sqldf(query2)
added_genres

,genre_id,genre_name
0,1,young adult
1,2,fiction
2,3,science fiction
3,4,dystopia
4,5,fantasy
...,...,...
1086,1087,nanotechnology
1087,1088,women
1088,1089,paranormal-and-fantasy
1089,1090,cooking-history


In [ ]:
# save the modified genre
added_genres.to_csv('/content/drive/My Drive/CIS550 Final Project/genres.csv')

In [ ]:
# replace the genre names in genre_df with genre id

query = """
select author_genre_signle.author_id, added_genres.genre_id from 
author_genre_signle left join added_genres on author_genre_signle.author_genre = added_genres.genre_name
where author_genre_signle.author_genre <> ""
"""

# schema: book_id, genre_id
auhor_has_genre = ps.sqldf(query)
auhor_has_genre

,author_id,genre_id
0,1,2
1,1,868
2,2,2
3,4,869
4,5,2
...,...,...
9654,31981,877
9655,32016,873
9656,32016,877
9657,32017,873


In [ ]:
# save the bookHasGenre dataset
auhor_has_genre.to_csv('/content/drive/My Drive/CIS550 Final Project/author_has_genre.csv')

## Part 4: Quotes Data Cleaning


###4.1 Cleaning data on Kaggle 
Cleaning and processing on original dataset have already been done at: https://www.kaggle.com/esteechenyiwei/get-quotes-from-sqlite/code 

(or in the final report: get_quotes_from_sqlite.ipynb)

###4.2 Matching books and adding bookID

In [ ]:
quotes_df = pd.read_csv('/content/drive/My Drive/CIS550 Final Project/final cleaned dataset/quotes.csv', header=0)
quotes_df = quotes_df[['id', 'content', 'author', 'book']]
quotes_df

,id,content,author,book
0,1,“My life amounts to no more than one drop in a...,David Mitchell,Cloud Atlas
1,2,“A half-read book is a half-finished love affa...,David Mitchell,Cloud Atlas
2,3,“Our lives are not our own. We are bound to ot...,David Mitchell,Cloud Atlas
3,4,"“Travel far enough, you meet yourself.”",David Mitchell,Cloud Atlas
4,5,"“Books don't offer real escape, but they can s...",David Mitchell,Cloud Atlas
...,...,...,...,...
348080,348081,"“It's going to look pretty good, then, isn't i...",Terry Pratchett,Sourcery
348081,348082,“It was sad music. But it waved its sadness li...,Terry Pratchett,Soul Music
348082,348083,“The merest accident of microgeography had mea...,Terry Pratchett,Small Gods
348083,348084,"“Look, that's why there's rules, understand? S...",Terry Pratchett,Thief of Time


In [ ]:
bookid_df = grdf_cleaned[['title', 'id']]

In [ ]:
query = '''SELECT DISTINCT quotes_df.id AS quotes_id, bookid_df.id AS book_id, title, author, quotes_df.content FROM bookid_df 
           LEFT JOIN quotes_df ON quotes_df.book = bookid_df.title'''
quotes_has_book = ps.sqldf(query).dropna()

In [ ]:
quotes_has_book

,quotes_id,book_id,title,author,content
0,145306.0,1,The Hunger Games,Suzanne Collins,"“If Peeta and I were both to die, or they thou..."
1,145309.0,1,The Hunger Games,Suzanne Collins,“Peeta opens his mouth for the first bite with...
2,168289.0,1,The Hunger Games,Suzanne Collins,“by Tim O'Brien Cover design by Elizabeth B. P...
3,83883.0,2,Harry Potter and the Order of the Phoenix,J.K. Rowling,"“You should have told her differently,' said H..."
4,123636.0,2,Harry Potter and the Order of the Phoenix,J.K. Rowling,“Just because you have the emotional range of ...
...,...,...,...,...,...
528599,221739.0,54296,Running the Books: The Adventures of an Accide...,Avi Steinberg,“In his eagerness to advance onto this new pat...
528600,221740.0,54296,Running the Books: The Adventures of an Accide...,Avi Steinberg,“but that didn’t stop him from going to church...
528601,221741.0,54296,Running the Books: The Adventures of an Accide...,Avi Steinberg,“He had no use for guns—these were for people ...
528602,221742.0,54296,Running the Books: The Adventures of an Accide...,Avi Steinberg,"“Though, from my perspective, it was almost ch..."


In [ ]:
q = '''SELECT * FROM quotes_has_book WHERE quotes_id = 1'''
ps.sqldf(q)

,quotes_id,book_id,title,author,content
0,1.0,399,Cloud Atlas,David Mitchell,“My life amounts to no more than one drop in a...
1,1.0,13067,Cloud Atlas,David Mitchell,“My life amounts to no more than one drop in a...
2,1.0,21640,Cloud Atlas,David Mitchell,“My life amounts to no more than one drop in a...
3,1.0,46354,Cloud Atlas,David Mitchell,“My life amounts to no more than one drop in a...


^I was a bit confused why there were even more rows after merging - and I realized that it's because there could be multiple editions of the same book, shown through different book_ids of the same title/author combination. 

## Part 5: Saving all dataframes

In [ ]:
grdf_cleaned = grdf_cleaned.drop(['genres', 'authors'], axis=1)


Index(['description', 'page_num', 'rating', 'rating_num', 'review_num',
       'title', 'image_url', 'id'],
      dtype='object')

In [ ]:
def f(s):
  if pd.isnull(s):
    return 0
  else:
    return int((str(s).split(" "))[0]) 

grdf_cleaned['page_num'] = grdf_cleaned['page_num'].apply(lambda x: f(x))


In [ ]:
grdf_cleaned

,description,page_num,rating,rating_num,review_num,title,image_url,id
0,Winning will make you famous. Losing means cer...,374,4.33,5519135,160706,The Hunger Games,https://images.gr-assets.com/books/1447303603l...,1
1,There is a door at the end of a silent corrido...,870,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,https://images.gr-assets.com/books/1255614970l...,2
2,The unforgettable novel of a childhood in a sl...,324,4.27,3745197,79450,To Kill a Mockingbird,https://images.gr-assets.com/books/1361975680l...,3
3,«È cosa ormai risaputa che a uno scapolo in po...,279,4.25,2453620,54322,Pride and Prejudice,https://images.gr-assets.com/books/1320399351l...,4
4,About three things I was absolutely positive.F...,498,3.58,4281268,97991,Twilight,https://images.gr-assets.com/books/1361039443l...,5
...,...,...,...,...,...,...,...,...
54296,"In this fearless and half-crazy story, Howard ...",256,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,https://images.gr-assets.com/books/1312074392l...,54297
54297,From the icons of the game to the players who ...,256,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",https://images.gr-assets.com/books/1348841629l...,54298
54298,NaN,0,3.66,32,3,"Wilpon's Folly - The Story of a Man, His Fortu...",https://images.gr-assets.com/books/1394277097l...,54299
54299,"Soon to be a major motion picture, from Brad P...",272,3.82,867,187,He Wanted the Moon: The Madness and Medical Ge...,https://images.gr-assets.com/books/1403192135l...,54300


In [ ]:
# reorder columns 
cols = ['id', 'title', 'description', 'page_num', 'rating', 'rating_num', 'review_num',
       'image_url']
grdf_cleaned = grdf_cleaned.reindex(columns=cols)

In [ ]:
grdf_cleaned

,id,title,description,page_num,rating,rating_num,review_num,image_url
0,1,The Hunger Games,Winning will make you famous. Losing means cer...,374,4.33,5519135,160706,https://images.gr-assets.com/books/1447303603l...
1,2,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,870,4.48,2041594,33264,https://images.gr-assets.com/books/1255614970l...
2,3,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,324,4.27,3745197,79450,https://images.gr-assets.com/books/1361975680l...
3,4,Pride and Prejudice,«È cosa ormai risaputa che a uno scapolo in po...,279,4.25,2453620,54322,https://images.gr-assets.com/books/1320399351l...
4,5,Twilight,About three things I was absolutely positive.F...,498,3.58,4281268,97991,https://images.gr-assets.com/books/1361039443l...
...,...,...,...,...,...,...,...,...
54296,54297,Taking the Field: A Fan's Quest to Run the Tea...,"In this fearless and half-crazy story, Howard ...",256,3.37,27,9,https://images.gr-assets.com/books/1312074392l...
54297,54298,"The Baseball Talmud: Koufax, Greenberg, and th...",From the icons of the game to the players who ...,256,3.97,34,5,https://images.gr-assets.com/books/1348841629l...
54298,54299,"Wilpon's Folly - The Story of a Man, His Fortu...",NaN,0,3.66,32,3,https://images.gr-assets.com/books/1394277097l...
54299,54300,He Wanted the Moon: The Madness and Medical Ge...,"Soon to be a major motion picture, from Brad P...",272,3.82,867,187,https://images.gr-assets.com/books/1403192135l...


In [ ]:
grdf_cleaned.to_csv('/content/drive/My Drive/CIS550 Final Project/books.csv')

In [ ]:
quotes_has_book.to_csv('/content/drive/My Drive/CIS550 Final Project/quotes_id.csv')